# Hill-type muscle model#
This notebook demonstrates the Hill-type muscle model. It builds on the openmuscle library by Phylliida Dev (not sure if this is a nickname or not) hosted here: https://github.com/Phylliida/openmuscle, which implements in Python the model of Daniel Haeufle [https://github.com/daniel-haeufle/macroscopic-muscle-model] and colleagues. I made a couple of minor modifications to `openmuscle` to return more of the state variables.

I also added the excitation-to-activation calculation based on https://simtk-confluence.stanford.edu/display/OpenSim/First-Order+Activation+Dynamics, and changed the activation and inactivation constants to approximate the curves shown in Robertson, G., Caldwell, G., Hamill, J., Kamen, G., & Whittlesey, S. (2013). *Research methods in biomechanics*, 2E. Human Kinetics.

In [1]:
import numpy as np
import openmuscle
from scipy.integrate import odeint

from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.gridspec as gridspec
import seaborn as sns
sns.set_style('white')
from matplotlib.colors import ListedColormap

/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# for generating activation from excitation
def get_u(t, pulse_amplitude, pulse_duration):
    '''
    Generates excitation at time points in `t`
    '''
    pulse_start = 0.2
    u = pulse_amplitude if (t > pulse_start) & (t < (pulse_start + pulse_duration)) else 0
    return u

def tau(a, u):
    '''
    See https://simtk-confluence.stanford.edu/display/OpenSim/First-Order+Activation+Dynamics
    
    Time constants altered to look similar to plots in 
    Robertson, G., Caldwell, G., Hamill, J., Kamen, G., & Whittlesey, S. (2013).
    Research methods in biomechanics, 2E. Human Kinetics.
    '''
    t_act   = 0.001   # s
    t_deact = 0.080   # s
    return t_act * (0.5 + 1.5 * a) if u > a else t_deact / (0.5 + 1.5 * a)

def dydt(y, t, pulse_amplitude, pulse_duration):
    '''
    Define differential equation relating activation and excitation.
    '''
    a = y
    u = get_u(t, pulse_amplitude, pulse_duration)
    dydt = (u - a) / tau(a, u)
    return dydt

In [3]:
def do_it(pulse_duration=0.005, pulse_amplitude=1, total_duration=1.0):
    dt = 0.0001
    a0 = 0
    t = np.arange(total_duration/dt) * dt
    a = odeint(dydt, a0, t, args=(pulse_amplitude, pulse_duration))
    u = [get_u(x, pulse_amplitude, pulse_duration) for x in t]
    
    v_ce   = np.zeros_like(t)
    f_mtc  = np.zeros_like(t)
    f_ce   = np.zeros_like(t)
    f_isom = np.zeros_like(t)
    f_see  = np.zeros_like(t)
    l_see  = np.zeros_like(t)
    muscle = openmuscle.Muscle(a[0])
    for i, ai in enumerate(a):
        f_mtc[i], f_ce[i], f_isomi, f_see[i], l_see[i] = muscle.step(ai, dt)
        f_isom[i] = f_isomi * ai
        v_ce[i] = muscle.v_CE
        
    fig = plt.figure(figsize=(12, 5))
    gs = gridspec.GridSpec(2, 3, height_ratios=[3,1])
    ax0 = plt.subplot(gs[0,0])
    ax1 = plt.subplot(gs[1,0])
    ax2 = plt.subplot(gs[:,1])
    ax3 = plt.subplot(gs[:,2])

    # allow time for model to settle at start
    s = slice(int(0.1/dt), None)

    ax0.plot(t[s], f_mtc[s] / muscle.CE.F_max, 'g-', label='$\mathsf{F_{MTC}}$')
    ax0.plot(t[s], f_isom[s], 'k-', label="$\mathsf{F_{isom}}$")
    ax0.legend(fontsize=14)

    ax1.plot(t[s], u[s], 'r-', label='excitation')
    ax1.plot(t[s], a[s], 'k-', label='activation')
    ax1.set_xlabel('time (s)')
    ax1.legend(fontsize=14)

    cmap = ListedColormap(sns.color_palette("Blues_d"))
    ax2.scatter(-v_ce[s], f_ce[s] / muscle.CE.F_max, c=t[s], vmin=t[0], vmax=t[-1], s=2, cmap=cmap)
    ax2.set_xlabel('CE velocity')
    ax2.set_ylabel('CE force')
    
    ax3.scatter(l_see[s], f_see[s], c=t[s], vmin=t[0], vmax=t[-1], s=2, cmap=cmap)
    ax3.set_xlabel('SEE length')
    ax3.set_ylabel('SEE force')
    
    sns.despine(fig)

_ = interact(do_it, pulse_duration=FloatSlider(value=0.2, min=0.0, max=0.9, step=0.05), \
             pulse_amplitude=FloatSlider(value=1, min=0, max=1), \
             total_duration=FloatSlider(value=1, min=0.5, max=2))

interactive(children=(FloatSlider(value=0.2, description='pulse_duration', max=0.9, step=0.05), FloatSlider(va…